# Objective

- Explain MEV through transactions in the wild 
- Discuss atomic arbitrage
- Submit bundles to flashbots

# GUI interaction

- Go to MEV explore [here](https://explore.flashbots.net/)
- Mostly covers arbitrage and liquidations, does not cover multi-tx MEV such as a sandwich attack

The folowing are some examples of MEV transactions.

## Arbitrage transaction
Exploiting price differences to make riskless profit – is usually benign.

- Search two or more exchanges with difference in asset prices
- Trade asset between them to gain tokens in the numeraire
- Consider an example of an arbitrage transaction [here](https://etherscan.io/tx/0xb55d4267a3565fdc8bada2638f97ed0bb31aa40bf8d4b304086dbdc1ca7d7844`)

## Liquidation transaction 
Like "margin calls" in traditional finance. Here, the miner spots lending positions that are underwater and liquidates them to earn a commission. Liquidation involves notifying the lending protocol that a the value of "collateral" posted by borrower has fallen. This makes the protocol take possession of the collateral and give a commission to the liquidator.

An example of a liquidation is shown [here](https://etherscan.io/tx/0xd70b42daec5bb9ac6e5df3d25d309f186db50df701f667e1f20b22448ea27d41).

## Sandwich transaction
Sandwiching is the act introducing transactions before and after user swaps to earn risk-free profit. Frontrunning (insertion before) is bad for the user and malicious. Backrunning (insertion after) is benign since it does not give anyone a worse price and stabilizes market.

- Frontrun a transaction to buy asset for cheap
- Let transactions incur
- Sell back to the original asset
- Consider the transactions for a multilayered sandwich [here](https://etherscan.io/txs?block=12775690&p=3) - courtesy @bertcmiller
![](img/multi_layered_sandwich1.png)

## JIT liquidity transaction

- Search for high value transaction
- Provide liquidity right before that transaction
- Earn fees
- Withdraw liquidity post the trade
- Consider the transactions for a JIT liquidity transaction [here](https://etherscan.io/txs?block=13601096&p=6) - courtesy @bertcmiller
![](img/JIT_liquidity1.png)

# Recall : Flashbots

Flashbots is a research and development organization that focuses on mitigating the negative impacts of MEV in DeFi networks, particularly on the Ethereum network. In particular, it started off by pointing out how MEV is a centralizing force in DeFi and proposed several products that are used by Ethereum nodes today to mitigate MEV.

They have developed a framework that allows users to send transaction bundles directly to miners through a "MEV-geth" node instead of submitting them to the open transaction pool, where they could be front-run or back-run. Miners can then extract the MEV from these transaction bundles without disrupting the overall network. Flashbots also provides a suite of tools and resources for developers and researchers to better understand and analyze MEV-related issues in DeFi.

## Downsides of priority gas auctions : example

[A 3-hop arbitrage]((https://etherscan.io/tx/0x2bde6e654eb93c990ae5b50a75ce66ef89ea77fb05836d7f347a8409f141599f)) between the 3 liquidity pools Balancer ETH/USDC, Sushiswap USDC/SIL and Sushiswap SIL/ETH that paid 12 ETH (USD 14.1k at the time) in gas fees and worth 16.7 ETH (USD 19.6k at the time) of Extracted MEV.

The high transaction fee hints at the fact that the sender iteratively bid up the gas price of their arbitrage transaction by replacing their existing trade before it gets mined with one with the same nonce but a higher gas price in order to compete with other traders going for the same opportunity. 

[One loser of the gas auction landed here](https://etherscan.io/tx/0x1ff41966cc02a4d8f6f1d80a6f49851e710b743d7c0ed143498608c0f421ec28) for a loss of 1.18 ETH. The winning transaction above ended up paying a gas price of 51.6k gwei, 547x higher than the average gas price at the time (94 gwei), but only 2x higher than the loser at 27.6k gwei.

- Extracted MEV breakdown for tx 1 - 4.7 ETH (USD 5.5k at the time) for the sender, 12 ETH (USD 14.1k at the time) for F2Pool, the miner of this block.
- Extracted MEV breakdown for tx 2 - 1.18 ETH (USD 1.4k at the time) for F2Pool, the miner of this block.

# Setup for flashbot bundles

- Install flashbots module for python in your notebook environment

Flashbots Provider Setup

In [1]:
!pip3 install web3
!pip3 install flashbots

In [4]:
# Set up a pipeline to create a bundle to send through flashbots
# Check references for flashbots here: https://docs.flashbots.net/flashbots-auction/searchers/quick-start
# Check references for eth_account here: https://readthedocs.org/projects/eth-account/downloads/pdf/latest/
import os
import secrets
from uuid import uuid4
from web3 import Web3, HTTPProvider
from eth_account import Account 
from eth_account.signers.local import LocalAccount
from flashbots import flashbot
from web3.exceptions import TransactionNotFound
from web3.types import TxParams

# Create a web3 object with a standard json rpc provider, such as Infura, Alchemy, or your own node.
w3 = Web3(HTTPProvider("https://goerli.infura.io/v3/4250fb4ff74c4fa5b27e19fa82451925"))

# signer is an Ethereum private key that does NOT store funds and is NOT your bot's primary key.
# This is an identifying key for signing payloads to establish reputation and whitelisting
os.environ['ETH_SIGNATURE_KEY'] = '0x' + secrets.token_hex(32)
signer: LocalAccount = Account.from_key(os.environ.get('ETH_SIGNATURE_KEY'))

# Flashbots providers require both a standard provider and ETH_ACCOUNT_SIGNATURE (to establish reputation)
flashbot(w3, signer, "https://relay-goerli.flashbots.net")

Generate a Transaction Bundle

In [8]:
# generate a random sender and receiver
os.environ['ETH_SENDER_KEY']  = 'ecf2392c4091906453c02ac4507fd7c557619b597ead5eed524e018c195a9b3e'
os.environ['ETH_RECEIVER_KEY'] =  '0x' + secrets.token_hex(32)
sender: LocalAccount = Account.from_key(os.environ.get('ETH_SENDER_KEY'))
receiver = Account.from_key(os.environ.get('ETH_RECEIVER_KEY')).address
print("sender address", sender.address)
print("receiver address", receiver)

# generate a transaction bundle with single or multiple transactions
nonce = w3.eth.get_transaction_count(sender.address)
tx1: TxParams = {
    "to": receiver,
    "value": Web3.toWei(0.001, "ether"),
    "gas": 21000,
    "maxFeePerGas": Web3.toWei(200, "gwei"),
    "maxPriorityFeePerGas": Web3.toWei(50, "gwei"),
    "nonce": nonce,
    "chainId": 5, # Goerli testnet chain id
    "type": 2,
}

tx1_signed = sender.sign_transaction(tx1)

tx2: TxParams = {
    "to": receiver,
    "value": Web3.toWei(0.001, "ether"),
    "gas": 21000,
    "maxFeePerGas": Web3.toWei(200, "gwei"),
    "maxPriorityFeePerGas": Web3.toWei(50, "gwei"),
    "nonce": nonce + 1,
    "chainId": 5, # Goerli testnet chain id
    "type": 2,
}

SINGLE_TX = False
if SINGLE_TX:
    bundle = [{"signed_transaction": tx1_signed.rawTransaction}]
else:
    bundle = [
        {"signed_transaction": tx1_signed.rawTransaction},
        {"signer": sender, "transaction": tx2},
    ]


sender address 0xDac66Ae3BEbB4aa9CC8d1BE815DAe2d2638A02a3
receiver address 0x454Ee341B7f34f11d202baF8B1cE63eC3b85dFfa


Send the Transaction Bundle to Flashbots Relayer

In [9]:
# keep trying to send bundle until it gets mined
while True:
    block = w3.eth.block_number
    print(f"Simulating on block {block}")
    # simulate bundle on current block
    try:
        w3.flashbots.simulate(bundle, block)
        print("Simulation successful.")
    except Exception as e:
        print("Simulation error", e)
        break 
    # send bundle targeting next block
    print(f"Sending bundle targeting block {block+1}")
    replacement_uuid = str(uuid4()) # generates a random unique ID
    print(f"replacementUuid {replacement_uuid}")
    # or send_bundle_munger
    send_result = w3.flashbots.send_bundle( 
        bundle,
        target_block_number=block + 1,
        opts={"replacementUuid": replacement_uuid},
    )
    print("bundleHash", w3.toHex(send_result.bundle_hash()))

    stats_v1 = w3.flashbots.get_bundle_stats(
        w3.toHex(send_result.bundle_hash()), block
    )
    print("bundleStats v1", stats_v1)

    stats_v2 = w3.flashbots.get_bundle_stats_v2(
        w3.toHex(send_result.bundle_hash()), block
    )
    print("bundleStats v2", stats_v2)

    send_result.wait()
    try:
        receipts = send_result.receipts()
        print(f"\nBundle was mined in block {receipts[0].blockNumber}\a")
        break
    except TransactionNotFound:
        print(f"Bundle not found in block {block+1}")
        # essentially a no-op but it shows that the function works
        cancel_res = w3.flashbots.cancel_bundles(replacement_uuid)
        print(f"canceled {cancel_res}")

print(
    f"Sender account balance: {Web3.fromWei(w3.eth.get_balance(sender.address), 'ether')} ETH"
)
print(
    f"Receiver account balance: {Web3.fromWei(w3.eth.get_balance(receiver), 'ether')} ETH"
)

Simulating on block 8597786
Simulation successful.
Sending bundle targeting block 8597787
replacementUuid cae41113-32fc-4ea6-a896-96cd2455cd6e
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isSimulated': False})
bundleStats v2 AttributeDict({'isSimulated': False})
Bundle not found in block 8597787
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597787
Simulation successful.
Sending bundle targeting block 8597788
replacementUuid 3c492943-ea47-4a22-8892-4fac9c8faacc
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:38:14.587Z', 'submittedAt': '2023-03-04T21:38:14.58Z'})
bundleStats v2 AttributeDict({'isSimulated': True, 'isHighPriority': True, 'simulatedAt': '2023-03-04T21:38:14.587Z', 'receivedAt': '2

Bundle not found in block 8597792
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597792
Simulation successful.
Sending bundle targeting block 8597793
replacementUuid c61e887e-cfa9-42ad-b902-0cfc2a83b1a9
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:39:26.632Z', 'submittedAt': '2023-03-04T21:39:26.627Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8c

bundleStats v2 AttributeDict({'isSimulated': True, 'isHighPriority': True, 'simulatedAt': '2023-03-04T21:39:26.632Z', 'receivedAt': '2023-03-04T21:39:26.627Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.707Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:05.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'times

Bundle not found in block 8597793
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597793
Simulation successful.
Sending bundle targeting block 8597794
replacementUuid 74ce0848-4790-4893-96fe-bbd55bcca0b9
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:39:39.062Z', 'submittedAt': '2023-03-04T21:39:39.056Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8c

bundleStats v2 AttributeDict({'isSimulated': True, 'isHighPriority': True, 'simulatedAt': '2023-03-04T21:39:39.062Z', 'receivedAt': '2023-03-04T21:39:39.056Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.707Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:05.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'times

Bundle not found in block 8597794
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597794
Simulation successful.
Sending bundle targeting block 8597795
replacementUuid 6e3abef9-0d23-4b13-ac7b-017a303a9c9f
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:39:49.542Z', 'submittedAt': '2023-03-04T21:39:49.537Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8c

Bundle not found in block 8597795
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597795
Simulation successful.
Sending bundle targeting block 8597796
replacementUuid e2a68b5d-a700-4d36-a34b-9fc3b8237f3c
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:40:05.032Z', 'submittedAt': '2023-03-04T21:40:05.026Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8c

Bundle not found in block 8597796
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597796
Simulation successful.
Sending bundle targeting block 8597797
replacementUuid 062c4b57-4e87-4658-ab7d-a32bb9e27974
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:40:27.55Z', 'submittedAt': '2023-03-04T21:40:27.545Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8ca

Bundle not found in block 8597797
canceled {'bundleHashes': ['0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529']}
Simulating on block 8597797
Simulation successful.
Sending bundle targeting block 8597798
replacementUuid 966f77ad-c126-42d0-b010-6bd1dcb35239
bundleHash 0xb59ef24859d19d2eb2a942d2cd3d986781f94be9031ce9b277841aa9ef645529
bundleStats v1 AttributeDict({'isHighPriority': True, 'isSentToMiners': False, 'isSimulated': True, 'simulatedAt': '2023-03-04T21:40:39.128Z', 'submittedAt': '2023-03-04T21:40:39.122Z', 'consideredByBuildersAt': [AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:03.708Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc', 'timestamp': '2023-03-04T21:39:04.207Z'}), AttributeDict({'pubkey': '0xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8c


Bundle was mined in block 8597798
Sender account balance: 0.26088880050546976 ETH
Receiver account balance: 0.002 ETH


# Additional reading

- Check out a nice explanation of sniping MEV exploits [here](https://www.samchepal.com/the-hidden-world-of/)